<a href="https://colab.research.google.com/github/shah-zeb-naveed/data-science-notes/blob/main/interview_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile lru_cache.py
import sys
import requests
from collections import OrderedDict

class ImageCache:
    def __init__(self, max_size_bytes):
        self.max_size = max_size_bytes
        self.cache = OrderedDict()
        self.current_size = 0

    def get(self, url):
        if url in self.cache:
            # Move to end to mark as recently used
            self.cache.move_to_end(url)
            return f"Cache Hit: {url}"
        else:
            try:
                response = requests.get(url)
                response.raise_for_status()
                image_bytes = response.content
                image_size = len(image_bytes)

                # If image is too big to cache, skip caching
                if image_size > self.max_size:
                    return f"Too big. Downloaded (Not Cached): {url}"

                # Evict least recently used items until there's space
                while self.current_size + image_size > self.max_size and self.cache:
                    _, old_bytes = self.cache.popitem(last=False) # first added
                    self.current_size -= len(old_bytes)

                self.cache[url] = image_bytes
                self.current_size += image_size
                return f"Downloaded and Cached: {url}"
            except requests.exceptions.RequestException as e:
                return f"Error downloading image: {e}"

def main():
    if len(sys.argv) < 2:
        print("Usage: python script.py <input_file>")
        sys.exit(1)

    # read fome file
    # input_file = sys.argv[1]
    # with open(input_file, 'r') as f:
    #     lines = f.read().splitlines()

    # max_size = int(lines[0])
    # num_urls = int(lines[1])
    # urls = lines[2:]

    # read from command line
    max_size = int(sys.argv[1])
    num_urls = int(sys.argv[2])
    urls = sys.argv[3:]

    cache = ImageCache(max_size)

    for url in urls:
        result = cache.get(url)
        print(result)

if __name__ == "__main__":
    main()

Overwriting lru_cache.py


In [ ]:
!python lru_cache.py 10000 3 https://placehold.co/600x400.png https://placehold.co/600x400.png https://placehold.co/600x400.png

Downloaded and Cached: https://placehold.co/600x400.png
Cache Hit: https://placehold.co/600x400.png
Cache Hit: https://placehold.co/600x400.png


In [ ]:
min({2:'A',1:'B'})

1

In [109]:
%%writefile advanced_lru_cache.py
import os
import sys
import requests
#import threading
import time
import pickle
from collections import OrderedDict, defaultdict, deque
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image
from heapq import heappush, heappop



class CacheItem:
    def __init__(self, url, content, size, timestamp, priority=0):
        self.url = url
        self.content = content
        self.size = size
        self.last_access = timestamp
        self.access_count = 1
        self.insert_time = timestamp
        self.priority = priority
        self.ttl_expiry = timestamp + 300  # default TTL of 5 minutes
        self.lfu_heap = []  # minheap for LFU
        self.fifo_queue = deque()  # for FIFO


class ImageCache:
    def __init__(self, max_size_bytes, eviction_policy="FIFO", max_item_size=10_000_000, bandwidth_limit=5, snapshot_file="cache_snapshot.pkl"):
        self.max_size = max_size_bytes
        self.max_item_size = max_item_size

        self.size = 0
#        self.lock = threading.RLock()
        self.eviction_policy = eviction_policy

        self.bandwidth_limit = bandwidth_limit  # max downloads per minute
        self.snapshot_file = snapshot_file

        self.cache = OrderedDict()
        self.download_timestamps = deque()
        self.load_snapshot()

    def save_snapshot(self):
        with open(self.snapshot_file, 'wb') as f:
            pickle.dump(self.cache, f)

    def load_snapshot(self):
        if os.path.exists(self.snapshot_file):
            with open(self.snapshot_file, 'rb') as f:
                self.cache = pickle.load(f)
                self.size = sum(item.size for item in self.cache.values())

    def _evict(self):

        # optimization
        if self.eviction_policy == "LFU":
            while self.lfu_heap:
                # insert_time not used but can be used for breaking ties
                access_count_at_push, _, url = heappop(self.lfu_heap)
                if url in self.cache:
                    item = self.cache[url]
                    # not stale if access_counts match
                    if item.access_count == access_count_at_push:
                        del self.cache[url]
                        self.size -= item.size
                        break  # valid eviction

        # practically, this is the same as ordereddict I think
        elif self.eviction_policy == "FIFO":
            while self.fifo_queue:
                url = self.fifo_queue.popleft()
                # evicted URLs can stay in queue so check this
                if url in self.cache:
                    item = self.cache.pop(url)
                    self.size -= item.size
                    break  # evicted

        if self.eviction_policy == "LRU":
            url, item = self.cache.popitem(last=False)
        elif self.eviction_policy == "FIFO":
            url = next(iter(self.cache))
            item = self.cache.pop(url)
        elif self.eviction_policy == "LFU":
            url = min(self.cache, key=lambda k: self.cache[k].access_count)
            item = self.cache.pop(url)

        elif self.eviction_policy == "PRIORITY":
            url = min(self.cache, key=lambda k: self.cache[k].priority)
            item = self.cache.pop(url)
        else:
            # default
            url, item = self.cache.popitem(last=False)

        self.size -= item.size

    def _is_bandwidth_limited(self):
        now = time.time()
        # remove timestamps older than 1 minute
        if self.download_timestamps:
          diff  = now - self.download_timestamps[0]
          #print('diff:', diff)

        # bandiwth measured per minute so remove any old before calculating numerator
        while self.download_timestamps and now - self.download_timestamps[0] > 60:
            self.download_timestamps.popleft()

        requests_per_minute = len(self.download_timestamps) + 1 # this current
        print('requests_per_minute:', requests_per_minute)
        return requests_per_minute > self.bandwidth_limit

    def _download(self, url):
        if self._is_bandwidth_limited():
            raise Exception(f"Bandwidth limit exceeded. Try again later.")

        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            content = response.content
            if len(content) > self.max_item_size:
                return f"Image too large to cache: {url}"
            self.download_timestamps.append(time.time())
            return content
        except requests.exceptions.RequestException as e:
            return f"Error downloading image: {e}"

    def get(self, url):
        #with self.lock:
          now = time.time()

          # Clean expired items
          # Eviction is a separate process than this
          expired = [key for key, item in self.cache.items() if now > item.ttl_expiry]
          for key in expired:
              self.size -= self.cache[key].size
              del self.cache[key]

          if url in self.cache:
              item = self.cache[url]
              item.last_access = now
              item.access_count += 1
              # Only move to end if we're using LRU policy
              if self.eviction_policy == "LRU":
                  self.cache.move_to_end(url)

              # Also inside get(), after item.access_count += 1, if LFU, push updated count
              if self.eviction_policy == "LFU":
                  heappush(self.lfu_heap, (item.access_count, item.insert_time, url))

              return f"CACHE HIT: {url}"

          result = self._download(url)

          image_size = len(result)
          if image_size > self.max_size:
              return f"Image exceeds total cache size: {url}"

          while self.size + image_size > self.max_size:
              self._evict()

          new_item = CacheItem(url, result, image_size, now)
          self.cache[url] = new_item
          self.size += image_size

          # optimizaiton
          if self.eviction_policy == "LFU":
              heappush(self.lfu_heap, (new_item.access_count + 1, new_item.insert_time, url))
          elif self.eviction_policy == "FIFO":
              self.fifo_queue.append(url)

          return f"DOWNLOADED: {url}"

    def prefetch(self, urls):
        for url in urls:
            #threading.Thread(target=self.get, args=(url,)).start()
            self.get(url)

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: python image_cache.py <input_file>")
        sys.exit(1)

    input_file = sys.argv[1]

    with open(input_file, 'r') as f:
        max_cache_size = int(f.readline().strip())
        n = int(f.readline().strip())
        urls = [f.readline().strip() for _ in range(n)]

    cache = ImageCache(max_cache_size, eviction_policy="FIFO")
    for url in urls:
        print('---')
        print(url)
        result = cache.get(url)
        print(result)
        #time.sleep(5)

    cache.get("https://placehold.co/600x400/EEE/31343C?font=poppins&text=Poppins")

    print('===')
    # Example of prefetching (next likely images)
    #cache.prefetch(["https://placehold.co/600x400/EEE/31343C?font=poppins&text=Poppins"])

    # Save snapshot at the end
    #cache.save_snapshot()

Overwriting advanced_lru_cache.py


In [110]:
%%writefile input.txt
10000
3
https://placehold.co/600x400.png
https://placehold.co/600x400.png
https://placehold.co/150x150.png

Overwriting input.txt


In [111]:
!python advanced_lru_cache.py input.txt

---
https://placehold.co/600x400.png
requests_per_minute: 1
DOWNLOADED: https://placehold.co/600x400.png
---
https://placehold.co/600x400.png
CACHE HIT: https://placehold.co/600x400.png
---
https://placehold.co/150x150.png
requests_per_minute: 2
DOWNLOADED: https://placehold.co/150x150.png
requests_per_minute: 3
===


| Feature                   | Heap-Based LFU                     | `min()` on `OrderedDict` LFU              |
| ------------------------- | ---------------------------------- | ----------------------------------------- |
| **Eviction Time**         | **O(log n)** (heap push/pop)       | **O(n)** (`min()` over all items)         |
| **Access Time (get/put)** | O(1) + heap push = O(log n)        | O(1) + O(1) update + O(n) min at eviction |
| **Space Overhead**        | O(n) for `cache`, O(n) for `heap`  | O(n) for `cache` only                     |
| **Pros**                  | Fast eviction                      | Simpler implementation                    |
| **Cons**                  | More memory and stale heap entries | Slower eviction due to full scan          |


| Feature            | Deque-Based FIFO                    | `OrderedDict` FIFO                 |
| ------------------ | ----------------------------------- | ---------------------------------- |
| **Eviction Time**  | O(1) with stale-check (amortized)   | **O(1)** via `popitem(last=False)` |
| **Access Time**    | O(1) + stale cleanup                | **O(1)**                           |
| **Space Overhead** | O(n) for `cache` + O(n) for `deque` | O(n) for single `OrderedDict`      |
| **Pros**           | Explicit queue logic, flexible      | Cleanest and most efficient        |
| **Cons**           | Potential stale entries             | Less customizable                  |


| Policy   | Best for Performance | Best for Simplicity |
| -------- | -------------------- | ------------------- |
| **LFU**  | Heap-based           | `min()` + `dict`    |
| **FIFO** | `OrderedDict`        | `OrderedDict`       |


| **Strategy**                         | **Pros**                                                                 | **Cons**                                                       | **Use Cases**                                                                    |
| ------------------------------------ | ------------------------------------------------------------------------ | -------------------------------------------------------------- | -------------------------------------------------------------------------------- |
| **LRU (Least Recently Used)**        | Simple and efficient eviction of least recently accessed items           | Poor for workloads with frequent full scans or cyclic patterns | Browser caches, database buffers, in-memory caches (e.g., `functools.lru_cache`) |
| **LFU (Least Frequently Used)**      | Retains hot items accessed repeatedly, good for skewed access patterns   | Requires frequency tracking; may retain stale "popular" items  | CDN edge caches, recommendation systems                                          |
| **FIFO (First In First Out)**        | Simple and predictable                                                   | Ignores access pattern; may evict useful items                 | Streaming data, write-behind caches                                              |
| **Random Replacement**               | Low-overhead eviction                                                    | Non-deterministic; can evict hot items                         | Hardware caches (e.g., CPU), when memory is tight                                |
| **TTL (Time to Live)**               | Auto-purges stale items after a time threshold                           | Fixed expiration may evict still-useful items                  | DNS caching, session tokens                                                      |
| **ARC (Adaptive Replacement Cache)** | Balances between LRU and LFU adaptively                                  | Complex to implement                                           | Filesystem page caching (ZFS, IBM DB2)                                           |
| **MRU (Most Recently Used)**         | Good for patterns where recently used items are less likely to be reused | Counterintuitive for many apps; rarely used alone              | Specific DB workloads (e.g., full table scans), specialized memory pools         |
| **2Q Cache**                         | Improves LRU by filtering out short-lived entries                        | Slightly more complex than LRU                                 | Web servers, virtual memory systems                                              |
| **Segmented LRU**                    | Prevents cache pollution from one workload segment                       | Needs workload classification                                  | JVM memory pools, multi-tenant environments                                      |


In [ ]:
%%writefile word_counter.py

import sys
from collections import defaultdict

def word_count(file_path):
    counts = defaultdict(int)
    with open(file_path, 'r') as f:
        for line in f:
            for word in line.strip().split():
                counts[word] += 1

    for word in sorted(counts):
        print(f"{word} {counts[word]}")

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python word_counter.py <input_file>")
        sys.exit(1)

    word_count(sys.argv[1])

Writing word_counter.py


In [ ]:
%%writefile input.txt
apple banana apple
orange apple banana

Writing input.txt


In [ ]:
!python word_counter.py input.txt

apple 3
banana 2
orange 1


In [ ]:
# prompt: create a short snippet that writes into csv file and then reads it back

import csv

def write_and_read_csv(filename, data):
    # Write data to CSV file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

def csv_reader(csvfile):
    # Read data from CSV file
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        read_data = list(reader)

    return read_data


# Example usage
data = [
    ["Name", "Age", "City"],
    ["Alice", "25", "New York"],
    ["Bob", "30", "Los Angeles"],
    ["Charlie", "28", "Chicago"]
]

filename = "example.csv"
read_data = write_and_read_csv(filename, data)
print(csv_reader(filename))

[['Name', 'Age', 'City'], ['Alice', '25', 'New York'], ['Bob', '30', 'Los Angeles'], ['Charlie', '28', 'Chicago']]
